In [67]:
from dash import dcc, Dash, dash_table, html
from src.db import Database
from dash.dependencies import Input
from dash.dependencies import Output
from itertools import chain
import pandas as pd

# Load database
database = Database()
database.load("data/databases/database__2025-03-21__17:12:35.196730.pkl")

# Parse tags
# db_exploded = database.db.explode("tag")
# db_exploded = db_exploded.rename(columns={"doi": "value", "tag": "label"})[["label", "value"]]
# tags = db_exploded.to_dict(orient="records")

# Pull out unique tags
# tags = database.db["tag"].unique().tolist()

# Create tag-DOI mapping
database.db["tag"] = database.db["tag"].apply(lambda lst: [s.lower() for s in lst])
db_exploded = database.db.explode("tag")
mapping = db_exploded.groupby("tag")["doi"].apply(list).to_dict()
tags = list(mapping.keys())

# tags = database.db[["doi", "tag"]]
database = database.db.drop('tag', axis=1)
database.columns = ["Title", "Authors", "Journal", "Year", "DOI"]

app = Dash()

app.layout = html.Div(
    children=[
        
        # Heading
        html.H1(children='A tagged database of active inference, predictive processing, and free energy principle papers'),
        
        # Tag filter menu
        dcc.Dropdown(
            id="filter_dropdown",
            # options=[{"label": tag, "value": tag} for tag in tags],
            options=tags,
            placeholder="Select a tag",
            multi=True,
            value=tags,
        ),
        
        # Download button
        
        # Main table
        dash_table.DataTable(
            id="database",
            columns=[
                {"name": i, "id": i} for i in database.columns],
            data=database.to_dict("records"),
            row_deletable=True,
            filter_action="native",
            sort_mode="multi",
            sort_action="native",
            style_header = {
                'text_align': 'left',
                'backgroundColor' : 'rgb(30, 144, 255)'
            },
            style_data={
                'whiteSpace': 'normal',
                'height': 'auto',
                'text_align': 'left'},
            style_data_conditional=[
                {'if': {'row_index': 'odd'},
                 'backgroundColor': 'rgb(240, 240, 240)'}
            ],
            style_cell_conditional=[
                {'if': {'column_id': 'Title'},
                 'width': '350px'},
                {'if': {'column_id': 'Authors'},
                 'width': '300px'},
                {'if': {'column_id': 'Year'},
                 'width': '100px'}],
            style_as_list_view=True
        )
    ]
)

@app.callback(
    Output("database", "data"), 
    Input("filter_dropdown", "value")
)
def display_table(tags):
    doi_list = [mapping[tag] for tag in tags]
    doi_list = list(chain.from_iterable(doi_list))
    doi_list = list(set(doi_list))
    
    filtered = database[database["DOI"].isin(doi_list)]
    return filtered.to_dict("records")

app.run(debug=True)

INFO:src.db:Database loaded from data/databases/database__2025-03-21__17:12:35.196730.pkl.
